In [206]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:

    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 len_numbers=6,
                 len_extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.len_numbers = len_numbers
        self.len_extra = len_extra

    @property
    def combo(self):
        return self._combo

    @property
    def extra(self):
        return self._extra

    @property
    def backend(self):
        return self._backend.__name__

    @property
    def many(self):
        return self._many

    @staticmethod
    def choice(len, max):
        numbers = list(range(1, max+1))

        def extraction():
            sample = rnd.choice(numbers)
            numbers.remove(sample)

            return sample
        return frozenset((extraction() for _ in range(len)))

    @staticmethod
    def sample(len, max):
        numbers = tuple(range(1, max))

        return frozenset(
            rnd.sample(numbers, k=len))

    def randint(self, _len, max):
        combo = set()
        numbers = iter(lambda: rnd.randint(1, max), 0)

        while len(combo) < _len:
            combo.add(next(numbers))

        return frozenset(combo)

    def manySamples(self):
        '''
        To add further randomness, this method simulates several extractions 
        and pick one of them casually, hopefully the winning one :D
        '''
        size = self._many or 1

        def sampler():
            stop = rnd.randint(1, size)
            samples = (self.extract() for _ in range(stop))

            for s in samples:
                sample = s

            return sample  # type: ignore
        return sampler

    def extract(self):
        combo = self._backend(self.len_numbers, self.max_numbers)
        extra = self._backend(self.len_extra, self.max_extra) or None

        return combo, extra

    def __call__(self, backend='choice', many=None):
        self._many = many
        self._backend = eval(backend,
                             {'__builtins__': {}},
                             {'choice': self.choice,
                              'randint': self.randint,
                              'sample': self.sample})

        self._combo, self._extra = self.manySamples()()

        return self

    @property
    def draw(self):
        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print('Estrazione del:', now, '\nNumeri Estratti:',
              *sorted(self._combo))  # type: ignore
        if self._extra is not None:
            print('Superstar:', *sorted(self._extra))  # type: ignore

In [207]:
superenalotto = Lotteria(max_numbers=90, max_extra=90, len_numbers=6, len_extra=1)
superenalotto(backend='choice', many=500_000).draw

Estrazione del: 24/06/2023 20:44 
Numeri Estratti: 7 19 20 23 53 59
Superstar: 60


In [208]:
eurojack = Lotteria(max_numbers=50, max_extra=12, len_numbers=5, len_extra=2)
eurojack(backend='sample', many=500_000).draw

Estrazione del: 24/06/2023 20:44 
Numeri Estratti: 2 10 11 41 49
Superstar: 1 7


In [209]:
winForLife = Lotteria(20, 20, 10, 1)
winForLife(backend='randint', many=100_000).draw

Estrazione del: 24/06/2023 20:44 
Numeri Estratti: 1 2 5 7 9 10 11 13 16 19
Superstar: 20


In [210]:
superenalotto.len_extra = 1

In [211]:
superenalotto(backend='randint').draw

Estrazione del: 24/06/2023 20:44 
Numeri Estratti: 6 11 22 29 48 64
Superstar: 62


In [212]:
superenalotto.backend

'randint'

In [213]:
superenalotto.__dict__

{'max_numbers': 90,
 'max_extra': 90,
 'len_numbers': 6,
 'len_extra': 1,
 '_many': None,
 '_backend': <bound method Lotteria.randint of <__main__.Lotteria object at 0x0000022E64952050>>,
 '_combo': frozenset({6, 11, 22, 29, 48, 64}),
 '_extra': frozenset({62})}

In [214]:
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b, many=0)
#    print('backend:', eurojack.backend)
#
# print('\n################# many ####################\n')
#
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b)
#    print('backend:', eurojack.backend)

In [215]:
win = Lotteria(20, 20, 10, 1)

w_1 = frozenset({2, 3, 5, 7, 9, 11, 12, 14, 16, 18})
w_2 = frozenset({7})

In [216]:
i = 1
win()
x_1, x_2 = win.combo, win.extra
while ((w_1 != x_1) and (not w_1.isdisjoint(x_1))) or (w_2 != x_2):
    win()
    x_1, x_2 = win.combo, win.extra
    i += 1
    if i % 1_000_000 == 0:
        print(i)
print(i, '>>>', x_1, x_2)

70894 >>> frozenset({2, 3, 5, 7, 9, 11, 12, 14, 16, 18}) frozenset({7})
